In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels as sm
from statsmodels.genmod.generalized_linear_model import GLM
import math
from geopy.distance import geodesic
from tqdm import tqdm
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point
import folium
import h3 #v4.3.0
import folium
import copy
#import county_tesslation

In [2]:
import os, sys, pathlib
os.environ["SPARK_HOME"] = "/opt/homebrew/opt/apache-spark/libexec"
os.environ["PATH"] = os.environ["SPARK_HOME"] + "/bin:" + os.environ["PATH"]
os.environ["PYSPARK_PYTHON"] = sys.executable


In [8]:
from sparkmobility.models.gravity import Gravity

In [ ]:
# reload
# import importlib
# import sparkmobility.models.gravity as gravity
# importlib.reload(gravity)
# from sparkmobility.models.gravity import Gravity

# H3 Resolution Setting

In [4]:
is_h3_hexagon = True
hex_resolution = 7
#hex_resolution = 6

# Loading Relevance Data

Various datasets can be used as sources of relevance data. The examples below show population data and stay count data, but other data can also be used.


## Population Info

In [ ]:
state_fips_codes=['06']
county_fips_codes={'06': ['001']}
#county_fips_codes={'06': ['001', '013', '041', '055', '075', '081', '085', '095', '097']} # Bay Area
year=2020
census_dataset='acs/acs5'
projection_crs='EPSG:3310'
census_variables=['NAME', 'B01003_001E']

In [ ]:
pop_df = Gravity.obtain_population_data(
    state_fips_codes, 
    county_fips_codes, 
    hex_resolution, 
    year,
    census_dataset, 
    projection_crs, 
    census_variables)
pop_df

,index,geometry,Hexagon Population
0,872830802ffffff,"POLYGON ((-122.31282 37.79273, -122.30530 37.8...",758.985504
1,872830803ffffff,"POLYGON ((-122.33797 37.80204, -122.33045 37.8...",2.979472
2,872830806ffffff,"POLYGON ((-122.31152 37.77041, -122.30401 37.7...",1476.979635
3,872830810ffffff,"POLYGON ((-122.26382 37.79642, -122.25629 37.8...",29596.097275
4,872830811ffffff,"POLYGON ((-122.28897 37.80574, -122.28144 37.8...",8329.553729
...,...,...,...
373,872836b6effffff,"POLYGON ((-121.68086 37.49044, -121.67326 37.5...",125.863471
374,872836b70ffffff,"POLYGON ((-121.58154 37.47478, -121.57393 37.4...",52.280754
375,872836b71ffffff,"POLYGON ((-121.60666 37.48431, -121.59905 37.4...",79.308014
376,872836b72ffffff,"POLYGON ((-121.55756 37.48768, -121.54994 37.4...",79.340645


## StayCount Info

In [ ]:
relevance_df = pd.read_parquet('./6-stays_h3_region.parquet')
# h3_id_region format: decimal to hexadecimal
relevance_df['h3_id_region'] = relevance_df['h3_id_region'].map(lambda x: hex(int(x))[2:])
relevance_df

,caid,h3_region_stay_id,stay_start_timestamp,stay_end_timestamp,stay_duration,h3_id_region,row_count_for_region
0,000124462fb7b671332bd72771fd49d126d7e1055d826a...,0,2019-01-01 01:00:14,2019-01-01 01:00:14,0,8929a40c8d3ffff,1
1,000124462fb7b671332bd72771fd49d126d7e1055d826a...,1,2019-01-05 18:02:09,2019-01-05 18:02:09,0,8929a428623ffff,1
2,000124462fb7b671332bd72771fd49d126d7e1055d826a...,2,2019-01-05 18:57:47,2019-01-05 18:57:47,0,8929a428693ffff,1
3,000124462fb7b671332bd72771fd49d126d7e1055d826a...,3,2019-01-05 19:26:12,2019-01-05 19:26:12,0,8929a40c8d3ffff,1
4,000124462fb7b671332bd72771fd49d126d7e1055d826a...,4,2019-01-11 19:55:48,2019-01-11 19:55:48,0,8929a42b673ffff,1
...,...,...,...,...,...,...,...
94180112,fffdd44510885e222e7dfbc4f308ae200ef6d2dcf8db8c...,36,2019-01-12 21:46:18,2019-01-12 21:46:18,0,892986b9087ffff,1
94180113,fffdd44510885e222e7dfbc4f308ae200ef6d2dcf8db8c...,37,2019-01-12 22:18:12,2019-01-12 22:18:55,43,892986b9543ffff,3
94180114,fffdd44510885e222e7dfbc4f308ae200ef6d2dcf8db8c...,38,2019-01-12 22:26:03,2019-01-12 22:26:03,0,892986b942fffff,1
94180115,fffe5ed3c9854f1f78baac9f60f658dd4900488fc96991...,0,2019-01-05 04:20:19,2019-01-05 04:20:29,10,8929124eb6bffff,2


In [11]:
stayCount_df = Gravity.obtain_relevance_data_at_resolution(
    relevance_df, 
    colname_h3_index='h3_id_region',
    colname_caid='caid',
    hex_resolution=hex_resolution)
stayCount_df

,index,stay_count,geometry
0,870c14666ffffff,1,"POLYGON ((-165.46328 64.52197, -165.47786 64.5..."
1,870c42c01ffffff,3,"POLYGON ((-149.43773 60.14067, -149.45497 60.1..."
2,870c42c05ffffff,1,"POLYGON ((-149.42660 60.16023, -149.44386 60.1..."
3,870c42c2bffffff,1,"POLYGON ((-149.36766 60.13496, -149.38492 60.1..."
4,870c51cc0ffffff,1,"POLYGON ((-151.50764 59.65959, -151.52404 59.6..."
...,...,...,...
88481,875d16b63ffffff,2,"POLYGON ((-156.34568 20.75491, -156.35010 20.7..."
88482,875d16b65ffffff,1,"POLYGON ((-156.33240 20.79403, -156.33683 20.7..."
88483,875d16b6cffffff,1,"POLYGON ((-156.30278 20.76417, -156.30720 20.7..."
88484,875d16b6dffffff,1,"POLYGON ((-156.27860 20.75729, -156.28303 20.7..."


In [ ]:
# (Optional) Filters `stayCount_df` to keep only rows with `index` values in `pop_df`.
valid_indices = set(pop_df['index'])
stayCount_df_sel = stayCount_df[
    stayCount_df['index'].isin(valid_indices)
]
stayCount_df_sel = stayCount_df_sel.reset_index(drop=True)
print(stayCount_df_sel.shape)
stayCount_df_sel.head()

(342, 3)


,index,stay_count,geometry
0,872830802ffffff,6612,"POLYGON ((-122.31282 37.79273, -122.30530 37.8..."
1,872830803ffffff,1572,"POLYGON ((-122.33797 37.80204, -122.33045 37.8..."
2,872830806ffffff,282,"POLYGON ((-122.31152 37.77041, -122.30401 37.7..."
3,872830810ffffff,113346,"POLYGON ((-122.26382 37.79642, -122.25629 37.8..."
4,872830811ffffff,26417,"POLYGON ((-122.28897 37.80574, -122.28144 37.8..."


# Loading Actual Flow Data

If no Actual Flow Data is available, you can use `Gravity.get_actual_flow_hex_df` to convert individual data (including `caid`, `origin`, `destination`, and `distance`) into Actual Flow Data (`flow_hex_df`). If Actual Flow Data already exists, you can simply load it directly.


## Conversion

In [ ]:
# actual_flow_df
actual_flow_df_path = 'trips' # includes columns: caid, origin, destination, distance
name_caid = 'caid'
name_origin = 'origin'
name_destination = 'destination'
name_distance = 'distance'

afdf = pd.read_parquet(actual_flow_df_path)
print(afdf.shape)
afdf.head()

(1854610, 4)


,caid,origin,destination,distance
0,00010baadaed40c9fbf9701ea443708cc1ee38b12a98a8...,8929a110a6fffff,8929a186ac3ffff,42.971812
1,00010baadaed40c9fbf9701ea443708cc1ee38b12a98a8...,8929a186ac3ffff,8929a1873abffff,7.815188
2,00010baadaed40c9fbf9701ea443708cc1ee38b12a98a8...,8929a1873abffff,8929a110a6fffff,38.607961
3,00010baadaed40c9fbf9701ea443708cc1ee38b12a98a8...,8929a110a6fffff,8929a110b4fffff,1.672450
4,00010baadaed40c9fbf9701ea443708cc1ee38b12a98a8...,8929a110b4fffff,8929a110a6fffff,1.672450


In [18]:
flow_hex_df = Gravity.get_actual_flow_hex_df(
    actual_flow_df_path,  
    name_caid, 
    name_origin, 
    name_destination, 
    name_distance,
    hex_resolution,
    pop_df, # filtering accoridng to pop_df range
    )
flow_hex_df

,origin,destination,flow,distance
0,872830802ffffff,872830803ffffff,2,0.917417
1,872830802ffffff,872830810ffffff,9,2.296542
2,872830802ffffff,872830811ffffff,11,1.670262
3,872830802ffffff,872830813ffffff,1,3.942312
4,872830802ffffff,872830814ffffff,1,4.431550
...,...,...,...,...
6712,872836b09ffffff,872836b0dffffff,1,1.069734
6713,872836b50ffffff,872836b09ffffff,1,2.444858
6714,872836b53ffffff,872836b50ffffff,1,2.456755
6715,872836b58ffffff,872836b5effffff,1,3.355534


## Direct Loading

In [ ]:
flow_hex_df = pd.read_parquet('../gravity/HomeWorkODMatrixR7')
print(flow_hex_df.shape)
flow_hex_df.head()

(2879, 4)


,origin,destination,flow,distance
0,8729a1d50ffffff,8729a1d62ffffff,1,9.422260
1,8729a0a6dffffff,8729a0b49ffffff,1,9.258245
2,8729a1d16ffffff,8729a56deffffff,1,20.606249
3,8729a5616ffffff,8729a56adffffff,1,6.780572
4,8729a56f3ffffff,8729a56d0ffffff,1,4.519659


# Modeling

## Model Setting

In [ ]:
# Model Setting
name = 'Example Gravity Model'
flow_data = flow_hex_df

gravity_type = 'single' #'single' 'global'
deterrence_func_type = 'power_law' #'power_law' 'exponential' 
relevance = 'stay' #'stay' 'pop'


if relevance == 'stay':
    relevance_df = stayCount_df_sel
    relevance_column = 'stay_count'
elif relevance == 'pop':
    relevance_df = pop_df 
    relevance_column = 'Hexagon Population'
# You can use other data as relecance.

linewidth_scale = 1 #0.0001


## Fitting and Generating

In [ ]:
# Fitting
# fit the parameters of the Gravity model from real flows

gravity_fitted = Gravity(
    name="Gravity model", 
    deterrence_func_type = deterrence_func_type, 
    deterrence_func_args=[-2.0], 
    origin_exp=1.0, 
    destination_exp=1.0, 
    gravity_type = gravity_type, 
    is_h3_hexagon=is_h3_hexagon
    )
print(gravity_fitted)

gravity_fitted.fit(
    flow_data, 
    relevance_df, 
    relevance_column,
    )
print(gravity_fitted)


# Generating
print('Generating...')
gravity_model_gen = Gravity(
    deterrence_func_type=deterrence_func_type,
    deterrence_func_args=gravity_fitted.get_str("deterrence_func_args")[0],
    origin_exp=gravity_fitted.get_str("origin_exp")[0],
    destination_exp=gravity_fitted.get_str("destination_exp")[0],
    gravity_type=gravity_type,
    name=name,
    is_h3_hexagon=is_h3_hexagon
)
'''

# Generating - No fitting
print('Generating...')
gravity_model_gen = gr.Gravity(
    deterrence_func_type=deterrence_func_type,
    deterrence_func_args=[-2.0],
    origin_exp=1.0,
    destination_exp=1.0,
    gravity_type=gravity_type,
    name=name,
    is_h3_hexagon=is_h3_hexagon
)
'''

np.random.seed(0)
flow_df = gravity_model_gen.generate(
    spatial_tessellation = relevance_df,
    tile_id_column='index',
    relevance_column = relevance_column,
    #tot_outflows_column='total_outflow',
    out_format='probabilities' # ‘flow', 
)

flow_df.to_csv(f'generative_flow_{relevance}_{gravity_type}_{deterrence_func_type}.csv')

print('Plotting...')

if gravity_type == 'global':
    linewidth_scale_mul = linewidth_scale * len(flow_df['origin'].unique())
elif gravity_type == 'single':
    linewidth_scale_mul = linewidth_scale

picFileName = f'flowMap_{relevance}_{gravity_type}_{deterrence_func_type}.png'

flow_df

Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-2.0], origin_exp=1.0, destination_exp=1.0, gravity_type="single")


100%|██████████| 6717/6717 [00:00<00:00, 52489.66it/s]

Fitting GLM Model...


Gravity(name="Gravity model", deterrence_func_type="power_law", deterrence_func_args=[-0.9875875692422995], origin_exp=1.0, destination_exp=0.5882093792426837, gravity_type="single")
Generating...


100%|██████████| 342/342 [00:04<00:00, 76.83it/s] 
/Users/hemingyi/Research/202410_skmob/sparkmobility-testing/sparkmobility/models/gravity.py:228: RuntimeWarning: divide by zero encountered in power
  self.deterrence_function = lambda d: d**exponent


Plotting...


,origin,destination,flow
0,872830802ffffff,872830803ffffff,0.002826
1,872830802ffffff,872830806ffffff,0.000500
2,872830802ffffff,872830810ffffff,0.115665
3,872830802ffffff,872830811ffffff,0.045549
4,872830802ffffff,872830812ffffff,0.032966
...,...,...,...
116617,872836b6affffff,872836b55ffffff,0.000006
116618,872836b6affffff,872836b58ffffff,0.000006
116619,872836b6affffff,872836b59ffffff,0.000081
116620,872836b6affffff,872836b5effffff,0.000010


# Visualization

In [23]:
flow_df['origin_lat'] = flow_df['origin'].apply(lambda h3_index: h3.cell_to_latlng(h3_index)[0])
flow_df['origin_lng'] = flow_df['origin'].apply(lambda h3_index: h3.cell_to_latlng(h3_index)[1])
flow_df['dest_lat'] = flow_df['destination'].apply(lambda h3_index: h3.cell_to_latlng(h3_index)[0])
flow_df['dest_lng'] = flow_df['destination'].apply(lambda h3_index: h3.cell_to_latlng(h3_index)[1])

In [24]:
df = flow_df

# Parameters for plotting thickness
# max_count = df[(df["count"] > 100) & (df["origin"] != df["destination"])]["count"].max() 
max_count = df[(df["origin"] != df["destination"])]["flow"].max() 
print(max_count)
min_thickness = 0.1 #minimum thickness to visualize
max_thickness = 10   #maximum thickness to visualize, corresponding to max_count / 50 * max_thickness 
factor = 5     #thickness = count/max * factor
cutoff_count = 0.02 #0.0001 #500  # Only visualize movement with count larger than this
max_circle_size = 30 #50



#Initiate flow_counter, for subsequent procedure of adding circles 
unique_destinations = df["destination"].unique()
unique_origins = df["origin"].unique()

# Combine the unique values from both columns into a single array
combined_unique_values = pd.unique(
    pd.concat([pd.Series(unique_destinations), pd.Series(unique_origins)])
)

# Create a Series with combined_unique_values as the index and initialize all values to 0
flow_counter = pd.Series(0, index=combined_unique_values, name="flow").astype(float)


map_center = [df['origin_lat'].mean(), df['origin_lng'].mean()]
m = folium.Map(location=map_center, zoom_start=10, width=800, height=500)

for idx, row in df.iterrows():
    start = (row['origin_lat'], row['origin_lng'])
    end = (row['dest_lat'], row['dest_lng'])
    thickness = max(min(row["flow"] / max_count * factor, max_thickness), min_thickness)
    if (row["flow"] > cutoff_count) & (start != end):
        flow_counter[row["origin"]] += thickness
        flow_counter[row["destination"]] += thickness
        folium.PolyLine(
            [start, end],
            color="blue",
            weight=thickness,  # Adjust divisor to 
            opacity=0.3
        ).add_to(m)

division = (flow_counter.max()/max_circle_size)

for index, item in flow_counter.items():
    if item > 0.1: 
        folium.CircleMarker(
            location = h3.cell_to_latlng(index),
            radius = item / division,
            weight=0,
            color='red',  # Adjust the color as needed
            fill=True,
            fill_color='red'
        ).add_to(m)
m

0.1633141452696307
